<a href="https://colab.research.google.com/github/quanganh1999/NF-Net-on-CIFAR/blob/main/Train_NF_ResNet_on_Cifar_10_using_PyTorch_Lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ⚙️ Imports and Setups

In [1]:
!nvidia-smi

Wed Apr 21 21:06:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd drive/MyDrive/NF-Resnet

/content/drive/MyDrive/NF-Resnet


In [4]:
%%capture
# Install pytorch lighting
!pip install pytorch-lightning --quiet
# Install weights and biases
!pip install wandb --quiet

In [5]:
!pip install lightning-bolts["extra"] --quiet

     |████████████████████████████████| 256kB 17.8MB/s 
     |████████████████████████████████| 37.6MB 80kB/s 
     |████████████████████████████████| 22.3MB 1.5MB/s 


In [ ]:
!git clone https://github.com/rwightman/pytorch-image-models

fatal: destination path 'pytorch-image-models' already exists and is not an empty directory.


In [ ]:
!pip install git+https://github.com/rwightman/pytorch-image-models.git --quiet

In [17]:
# regular imports
import sys
sys.path.append("pytorch-image-models")
import os
import re
import numpy as np

# pytorch related imports 
import torch
from torch import nn
from torch.nn import functional as F
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from torchvision.datasets.utils import download_url

# import for nfnet
import timm
from timm.utils import *
from timm.models import model_parameters

# lightning related imports
import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint

# sklearn related imports
from sklearn.metrics import precision_recall_curve
from sklearn.preprocessing import label_binarize

# import wandb and login
import wandb
wandb.login()

wandb: Currently logged in as: quanganhk62 (use `wandb login --relogin` to force relogin)


True

In [18]:
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.datasets import CIFAR10
from pytorch_lightning.utilities.seed import seed_everything
import pl_bolts

In [8]:
seed_everything(1999)

Global seed set to 1999


1999

## Create base model

In [27]:
base_model = timm.create_model("eca_nfnet_l1", pretrained=True) #256 input 320 test

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/ecanfnet_l1_ra2-7dce93cd.pth" to /root/.cache/torch/hub/checkpoints/ecanfnet_l1_ra2-7dce93cd.pth


In [28]:
config = base_model.default_cfg
config

{'architecture': 'eca_nfnet_l1',
 'classifier': 'head.fc',
 'crop_pct': 1.0,
 'first_conv': 'stem.conv1',
 'input_size': (3, 256, 256),
 'interpolation': 'bicubic',
 'mean': (0.485, 0.456, 0.406),
 'num_classes': 1000,
 'pool_size': (8, 8),
 'std': (0.229, 0.224, 0.225),
 'test_input_size': (3, 320, 320),
 'url': 'https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/ecanfnet_l1_ra2-7dce93cd.pth'}

## 🎨 Using DataModules - `Clatech101DataModule`

DataModules are a way of decoupling data-related hooks from the `LightningModule` so you can develop dataset agnostic models.

In [30]:
class CIFAR10Data(pl.LightningDataModule):
      def __init__(self, batch_size, data_dir: str = './'):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size        
        self.mean = (0.4914, 0.4822, 0.4465)
        self.std = (0.2471, 0.2435, 0.2616)

        #augmentation (use other strong augmentation)
        # self.train_transform = transforms.Compose(
        #     [
        #         transforms.RandomCrop(32, padding=4),
        #         transforms.RandomHorizontalFlip(),
        #         transforms.ToTensor(),
        #         transforms.Normalize(self.mean, self.std),
        #     ]
        # )

        #typical resize
        # self.train_transform = transforms.Compose([
        #   transforms.RandomResizedCrop((256, 256), scale=(0.05, 1.0)),
        #   transforms.ToTensor(),
        #   transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        # ])

        #timm resize
        self.train_transform = timm.data.transforms_factory.transforms_imagenet_train(
            img_size = 256,
            scale = (0.05, 1.0),
            interpolation=config["interpolation"],
            mean=config["mean"],
            std=config["std"]            
        )
   
        #need to resize for more acc ??
        # self.test_transform = transforms.Compose(
        #     [
        #         transforms.ToTensor(),
        #         transforms.Normalize(self.mean, self.std),
        #     ]
        # )

        #typical resize
        # self.test_transform = transforms.Compose([
        # transforms.Resize((320, 320)),
        # transforms.ToTensor(),
        # transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        # ])

        #timm resize
        self.test_transform = timm.data.transforms_factory.transforms_imagenet_eval(
            img_size = 320,
            interpolation=config["interpolation"],
            mean=config["mean"],
            std=config["std"],
            crop_pct=config["crop_pct"]
        )

        # self.dims = (3, 32, 32)
        self.num_classes = 10

      def prepare_data(self):
        # download 
        CIFAR10(self.data_dir, train=True, download=True)
        CIFAR10(self.data_dir, train=False, download=True)    

      def setup(self, stage=None):        
        if stage == 'fit' or stage is None:
            # load the dataset    
            self.cifar_full_train = CIFAR10(self.data_dir, train=True, transform=self.train_transform)
            self.cifar_full_val = CIFAR10(self.data_dir, train=True, transform=self.test_transform)             
            num_train = len(self.cifar_full_train)
            indices = list(range(num_train))
            split = int(np.floor(0.1 * num_train))
            np.random.seed(1999)
            np.random.shuffle(indices)

            #train
            train_idx, valid_idx = indices[split:], indices[:split]
            self.train_sampler = SubsetRandomSampler(train_idx)
            self.valid_sampler = SubsetRandomSampler(valid_idx)            

        # Assign test dataset for use in dataloader(s)
        if stage == 'test' or stage is None:
            self.cifar_test = CIFAR10(self.data_dir, train=False, transform=self.test_transform)        

      def train_dataloader(self):
        #return DataLoader(self.cifar_full_train, batch_size=self.batch_size, sampler=self.train_sampler)
        return DataLoader(self.cifar_full_train, batch_size=self.batch_size, shuffle=True)

      def val_dataloader(self):      
        #return DataLoader(self.cifar_full_val, batch_size=self.batch_size, sampler=self.valid_sampler)
        return DataLoader(self.cifar_test, batch_size=self.batch_size)

      def test_dataloader(self):
        return DataLoader(self.cifar_test, batch_size=self.batch_size)

## 📲 Callbacks

#### 🚏 Earlystopping

In [22]:
early_stop_callback = EarlyStopping(
   monitor='val_loss',
   patience=3,
   verbose=False,
   mode='min'
)

#### 🛃 Custom Callback - `ImagePredictionLogger`

In [23]:
class ImagePredictionLogger(Callback):
    def __init__(self, val_samples, num_samples=32):
        super().__init__()
        self.num_samples = num_samples
        self.val_imgs, self.val_labels = val_samples
        
    def on_validation_epoch_end(self, trainer, pl_module):
        val_imgs = self.val_imgs.to(device=pl_module.device)
        val_labels = self.val_labels.to(device=pl_module.device)
       
        logits = pl_module(val_imgs)
        preds = torch.argmax(logits, -1)
        
        trainer.logger.experiment.log({
            "examples":[wandb.Image(x, caption=f"Pred:{pred}, Label:{y}") 
                           for x, pred, y in zip(val_imgs[:self.num_samples], 
                                                 preds[:self.num_samples], 
                                                 val_labels[:self.num_samples])]
            })

#### 💾 Model Checkpoint Callback

In [24]:
MODEL_CKPT_PATH = './model1'
# MODEL_CKPT = 'model-{epoch:02d}-{val_loss:.2f}'
MODEL_CKPT = 'model1-{epoch:02d}-{val_acc:.3f}'

checkpoint_callback = ModelCheckpoint(
    monitor='val_acc',
    dirpath = MODEL_CKPT_PATH,
    filename=MODEL_CKPT,
    save_top_k=3,
    save_last = True,
    mode='max')

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: Checkpoint directory ./model1 exists and is not empty.
  warnings.warn(*args, **kwargs)


## 🎺 Define The Model

In [25]:
class LitModel(pl.LightningModule):
    def __init__(self, input_shape, num_classes, learning_rate=2e-4, used_agc = False, clip_grad = 0.01):
        super().__init__()
        
        # log hyperparameters
        self.save_hyperparameters()
        self.learning_rate = learning_rate
        self.dim = input_shape
        self.num_classes = num_classes

        #train from scratch
        # self.classifier = timm.create_model('nf_resnet50', pretrained=False, num_classes = num_classes)

        #transfer learning
        #fine-tuning
        self.model = timm.create_model("eca_nfnet_l1", pretrained=True, num_classes=num_classes) #256 input 320 test

        #feature exactor (freezee all layers except the last layer)
        # self.model = timm.create_model("eca_nfnet_l1", pretrained=True)        
        # for param in self.model.parameters():
        #     param.requires_grad = False
        # self.model.reset_classifier(num_classes)
    
    def forward(self, x):
      x = F.log_softmax(self.model.forward(x), dim=1)
      return x

    #using Adaptive Gradient Descent
    #adapt from train procedure of timm package: https://github.com/rwightman/pytorch-image-models/blob/master/train.py        
    #clip_grad: clipping factor
    def on_after_backward(self):
      if(self.used_agc):
        dispatch_clip_grad(model_parameters(self.model, exclude_head = True),
                           value = clip_grad, mode = "agc")

    # logic for a single training step
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        
        # training metrics
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)
        self.log('train_loss', loss, on_step=True, on_epoch=True, logger=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, logger=True)
        
        return loss

    # logic for a single validation step
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)

        # validation metrics
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
        return loss

    # logic for a single testing step
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        
        # validation metrics
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)
        self.log('test_loss', loss, prog_bar=True)
        self.log('test_acc', acc, prog_bar=True)
        return loss

    def configure_optimizers(self):
        # optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-6, nesterov = True)
        scheduler = pl_bolts.optimizers.lr_scheduler.LinearWarmupCosineAnnealingLR(optimizer, warmup_epochs = 5, max_epochs = 60)
        # return optimizer
        return [optimizer], [scheduler]


## ⚡ Train and Evaluate The Model

In [31]:
# Init our data pipeline
BATCH_SIZE = 256
dm = CIFAR10Data(batch_size=BATCH_SIZE)
# To access the x_dataloader we need to call prepare_data and setup.
dm.prepare_data()
dm.setup()

# Samples required by the custom ImagePredictionLogger callback to log image predictions.
val_samples = next(iter(dm.val_dataloader()))
val_imgs, val_labels = val_samples[0], val_samples[1]
val_imgs.shape, val_labels.shape

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:258: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Files already downloaded and verified
Files already downloaded and verified


(torch.Size([256, 3, 320, 320]), torch.Size([256]))

In [32]:
# Init our model
model = LitModel(input_shape=(3, 256, 256), num_classes = 10, used_agc=True, clip_grad=0.015)

# Initialize wandb logger
wandb_logger = WandbLogger(project='nfnet', job_type='train-nf-resnet')

# Initialize a trainer
trainer = pl.Trainer(max_epochs=60,
                     progress_bar_refresh_rate=5, 
                     gpus=1, 
                     logger=wandb_logger,
                     #early_stop_callback,                     
                     callbacks=[ImagePredictionLogger(val_samples), checkpoint_callback]
                     )

# Train the model ⚡🚅⚡
trainer.fit(model, dm) 

# Evaluate the model on the held out test set ⚡⚡
trainer.test()

# Close wandb run
wandb.finish()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



  | Name  | Type        | Params
--------------------------------------
0 | model | NormFreeNet | 38.4 M
--------------------------------------
38.4 M    Trainable params
0         Non-trainable params
38.4 M    Total params
153.462   Total estimated model params size (MB)


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional.py:365: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


RuntimeError: ignored

In [ ]:
#resume from checkpoint:
model = LitModel((3, 256, 256), 10)

# Initialize wandb logger
wandb_logger = WandbLogger(project='nfnet', job_type='train-nf-resnet')

# Initialize a trainer
trainer = pl.Trainer(max_epochs=60,
                     progress_bar_refresh_rate=5, 
                     gpus=1, 
                     logger=wandb_logger,
                     #early_stop_callback,
                     callbacks=[ImagePredictionLogger(val_samples), checkpoint_callback],
                     resume_from_checkpoint='model/last.ckpt'
                     )

# Train the model ⚡🚅⚡
trainer.fit(model, dm) 

# Evaluate the model on the held out test set ⚡⚡
trainer.test()

# Close wandb run
wandb.finish()

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/ecanfnet_l1_ra2-7dce93cd.pth" to /root/.cache/torch/hub/checkpoints/ecanfnet_l1_ra2-7dce93cd.pth
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: Currently logged in as: quanganhk62 (use `wandb login --relogin` to force relogin)



  | Name  | Type        | Params
--------------------------------------
0 | model | NormFreeNet | 38.4 M
--------------------------------------
38.4 M    Trainable params
0         Non-trainable params
38.4 M    Total params
153.462   Total estimated model params size (MB)
Restored states from the checkpoint file at model/last.ckpt


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
Saving latest checkpoint...


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.9818999767303467, 'test_loss': 0.06278768926858902}
--------------------------------------------------------------------------------


_runtime,321
_timestamp,1618879153
_step,3
train_loss_step,0.32634
train_acc_step,0.90625
epoch,20
trainer/global_step,15706
test_loss,0.06279
test_acc,0.9819


_runtime,▁▁▄█
_timestamp,▁▁▄█
_step,▁▃▆█
train_loss_step,█▁
train_acc_step,▁█
epoch,▁▁▁
trainer/global_step,▁▇█
test_loss,▁
test_acc,▁


In [ ]:
#Test with checkpoint
LitModel((3, 32, 32), 10)
model = LitModel.load_from_checkpoint(checkpoint_path='./model/model2-epoch=41-val_acc=0.974.ckpt')

# init trainer with whatever options
trainer = trainer = pl.Trainer(gpus=1)

# test (pass in the model)
trainer.test(model, datamodule = dm)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/ecanfnet_l1_ra2-7dce93cd.pth" to /root/.cache/torch/hub/checkpoints/ecanfnet_l1_ra2-7dce93cd.pth
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.973800003528595, 'test_loss': 0.09650633484125137}
--------------------------------------------------------------------------------


[{'test_acc': 0.973800003528595, 'test_loss': 0.09650633484125137}]

In [ ]:
%ls model/

 last.ckpt                            'model3-epoch=47-val_acc=0.971.ckpt'
'model1-epoch=08-val_acc=0.968.ckpt'  'model3-epoch=49-val_acc=0.968.ckpt'
'model1-epoch=09-val_acc=0.956.ckpt'  'model3-epoch=50-val_acc=0.968.ckpt'
'model1-epoch=10-val_acc=0.964.ckpt'  'model-epoch=34-val_acc=0.92.ckpt'
'model2-epoch=36-val_acc=0.974.ckpt'  'model-epoch=43-val_acc=0.90.ckpt'
'model2-epoch=37-val_acc=0.973.ckpt'  'model-epoch=45-val_acc=0.90.ckpt'
'model2-epoch=41-val_acc=0.974.ckpt'  'model-epoch=51-val_acc=0.92.ckpt'
'model3-epoch=43-val_acc=0.971.ckpt'  'model-epoch=55-val_acc=0.90.ckpt'
'model3-epoch=45-val_acc=0.971.ckpt'
